## Dataset Preparation

### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

#Ignoring warnings
import warnings
warnings.filterwarnings("ignore")

### Importing Target variable [ y-variable] (S&P Case-Schiller Home Price Index)

In [2]:
data = pd.read_csv('CSUSHPISA.csv')
data.head()

DATE  CSUSHPISA
0  1987-01-01     63.965
1  1987-02-01     64.424
2  1987-03-01     64.736
3  1987-04-01     65.132
4  1987-05-01     65.563

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   DATE       440 non-null    object 
 1   CSUSHPISA  440 non-null    float64
dtypes: float64(1), object(1)
memory usage: 7.0+ KB


* The datatype for DATE column is **object** , 
* It is converted to date-time format to join the other parameters

In [4]:
data.DATE = pd.to_datetime(data.DATE)

In [5]:
dataset_files = ['HNFSEPUSSA', 'HOUST1F', 'HSN1F', 'INTDSRUSM193N',
       'LFACTTTTUSM657S', 'MSACSR', 'NA000334Q', 'NASDAQCOM', 'PERMIT',
       'PERMIT1', 'QUSR628BIS', 'RSAHORUSQ156S', 'TTLCONS', 'UNRATE','CSUSHPISA']

In [6]:
# creating the function to convert the date time format of all the parameters to YYYY-MM-DD

def date_col(arr):
    arr = arr.str.replace(" ","-")
    for i in range(len(arr)):
        arr[i]=dt.datetime.strptime(arr[i], '%d-%m-%y').strftime('%Y-%m-%d')
    return arr

### Combining All Features ( x-variables )

In [7]:
# combining all the features into a dataset (**from files**)

for i in range(len(dataset_files)-1):
    data1 = pd.read_csv(dataset_files[i]+'.csv')
    if len(data1.iloc[0,0])==8:
        data1.iloc[:,0] = date_col(data1.iloc[:,0])
    data1.DATE = pd.to_datetime(data1.DATE)
    data = pd.merge(left=data, right=data1, left_on='DATE', right_on='DATE',how='left')

In [8]:
data.head()

DATE  CSUSHPISA  HNFSEPUSSA  HOUST1F  HSN1F  INTDSRUSM193N  \
0 1987-01-01     63.965       356.0     1224    709            5.5   
1 1987-02-01     64.424       357.0     1282    732            5.5   
2 1987-03-01     64.736       358.0     1205    713            5.5   
3 1987-04-01     65.132       358.0     1181    735            5.5   
4 1987-05-01     65.563       353.0     1141    651            5.5   

   LFACTTTTUSM657S  MSACSR  NA000334Q   NASDAQCOM  PERMIT  PERMIT1  \
0         0.197284     6.0  1148876.0  384.227143  1690.0   1088.0   
1         0.233077     6.2        NaN  411.712632  1689.0   1195.0   
2         0.124242     6.0        NaN  432.204545  1704.0   1132.0   
3         0.055337     6.0  1205023.0  422.771429  1601.0   1057.0   
4         0.563116     6.7        NaN  416.634000  1500.0   1006.0   

   QUSR628BIS  RSAHORUSQ156S  TTLCONS  UNRATE  
0     85.4172           63.9      NaN     6.6  
1         NaN            NaN      NaN     6.6  
2         NaN            NaN      NaN     6.6  
3     86.2084           63.9      NaN     6.3  
4         NaN            NaN      NaN     6.3

In [9]:
# selecting data from 2000 to 2020 As per condition

data = data[(data.DATE.dt.year>=2000) & (data.DATE.dt.year<=2020)]

In [10]:
data.reset_index(drop=True,inplace=True)

In [11]:
data.head()

DATE  CSUSHPISA  HNFSEPUSSA  HOUST1F  HSN1F  INTDSRUSM193N  \
0 2000-01-01    100.551       311.0     1268    873           5.00   
1 2000-02-01    101.339       299.0     1255    856           5.24   
2 2000-03-01    102.127       313.0     1313    900           5.34   
3 2000-04-01    102.922       305.0     1275    841           5.50   
4 2000-05-01    103.678       305.0     1230    857           5.71   

   LFACTTTTUSM657S  MSACSR  NA000334Q    NASDAQCOM  PERMIT  PERMIT1  \
0         1.490972     4.3  2448892.0  4013.494000  1727.0   1277.0   
1         0.132849     4.3        NaN  4410.871000  1692.0   1241.0   
2        -0.015443     4.3        NaN  4802.988261  1651.0   1253.0   
3         0.222559     4.4  2569266.0  3863.637368  1597.0   1192.0   
4        -0.254289     4.4        NaN  3528.416818  1543.0   1182.0   

   QUSR628BIS  RSAHORUSQ156S   TTLCONS  UNRATE  
0     95.3071           67.1  784940.0     4.0  
1         NaN            NaN  793737.0     4.1  
2         NaN            NaN  809459.0     4.0  
3     96.7197           67.3  804766.0     3.8  
4         NaN            NaN  805005.0     4.0

### Checking Null Values

In [12]:
data.isna().sum()

DATE                 0
CSUSHPISA            0
HNFSEPUSSA           0
HOUST1F              0
HSN1F                0
INTDSRUSM193N        0
LFACTTTTUSM657S      0
MSACSR               0
NA000334Q          168
NASDAQCOM            0
PERMIT               0
PERMIT1              0
QUSR628BIS         168
RSAHORUSQ156S      168
TTLCONS              0
UNRATE               0
dtype: int64

## Treatment on Null values

In [13]:
# Filling null values for features which has quaterly frquency

data['NA000334Q'] = data['NA000334Q'].fillna(method='ffill')
data['QUSR628BIS'] = data['QUSR628BIS'].fillna(method='ffill')
data['RSAHORUSQ156S'] = data['RSAHORUSQ156S'].fillna(method='ffill')

In [14]:
# rearranging
data = data[["DATE"]+dataset_files]
data.head()

DATE  HNFSEPUSSA  HOUST1F  HSN1F  INTDSRUSM193N  LFACTTTTUSM657S  \
0 2000-01-01       311.0     1268    873           5.00         1.490972   
1 2000-02-01       299.0     1255    856           5.24         0.132849   
2 2000-03-01       313.0     1313    900           5.34        -0.015443   
3 2000-04-01       305.0     1275    841           5.50         0.222559   
4 2000-05-01       305.0     1230    857           5.71        -0.254289   

   MSACSR  NA000334Q    NASDAQCOM  PERMIT  PERMIT1  QUSR628BIS  RSAHORUSQ156S  \
0     4.3  2448892.0  4013.494000  1727.0   1277.0     95.3071           67.1   
1     4.3  2448892.0  4410.871000  1692.0   1241.0     95.3071           67.1   
2     4.3  2448892.0  4802.988261  1651.0   1253.0     95.3071           67.1   
3     4.4  2569266.0  3863.637368  1597.0   1192.0     96.7197           67.3   
4     4.4  2569266.0  3528.416818  1543.0   1182.0     96.7197           67.3   

    TTLCONS  UNRATE  CSUSHPISA  
0  784940.0     4.0    100.551  
1  793737.0     4.1    101.339  
2  809459.0     4.0    102.127  
3  804766.0     3.8    102.922  
4  805005.0     4.0    103.678

In [15]:
data.isnull().sum()

DATE               0
HNFSEPUSSA         0
HOUST1F            0
HSN1F              0
INTDSRUSM193N      0
LFACTTTTUSM657S    0
MSACSR             0
NA000334Q          0
NASDAQCOM          0
PERMIT             0
PERMIT1            0
QUSR628BIS         0
RSAHORUSQ156S      0
TTLCONS            0
UNRATE             0
CSUSHPISA          0
dtype: int64

### Adding some more Feature

In [16]:
df = pd.read_csv('historical us home prices.csv')
df.head()

category  Median Home Price (NSA)  CPI-Adjusted Price
0  Thu Jan 15 1953                 18080.26           208688.34
1  Sun Feb 15 1953                 18080.26           209475.85
2  Sun Mar 15 1953                 18080.26           208688.34
3  Wed Apr 15 1953                 18122.25           209173.00
4  Fri May 15 1953                 18143.25           208631.07

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 847 entries, 0 to 846
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   category                 847 non-null    object 
 1   Median Home Price (NSA)  847 non-null    float64
 2   CPI-Adjusted Price       847 non-null    float64
dtypes: float64(2), object(1)
memory usage: 20.0+ KB


In [18]:
for i in range(len(df)):
    df.iloc[i,0] = df.iloc[i,0][4:] 

In [19]:
df.head()

category  Median Home Price (NSA)  CPI-Adjusted Price
0  Jan 15 1953                 18080.26           208688.34
1  Feb 15 1953                 18080.26           209475.85
2  Mar 15 1953                 18080.26           208688.34
3  Apr 15 1953                 18122.25           209173.00
4  May 15 1953                 18143.25           208631.07

In [20]:
# formatting date into desired format

for i in range(len(df)):
    df.iloc[i,0]=dt.datetime.strptime(df.iloc[i,0], '%b %d %Y').strftime('%Y-%m-%d')

In [21]:
df.head()

category  Median Home Price (NSA)  CPI-Adjusted Price
0  1953-01-15                 18080.26           208688.34
1  1953-02-15                 18080.26           209475.85
2  1953-03-15                 18080.26           208688.34
3  1953-04-15                 18122.25           209173.00
4  1953-05-15                 18143.25           208631.07

In [22]:
# resetting the date to start of each month to ease the merging process
df.iloc[:,0] = df.iloc[:,0].str.replace("-15","-01")

In [23]:
df.head(12)

category  Median Home Price (NSA)  CPI-Adjusted Price
0   1953-01-01                 18080.26           208688.34
1   1953-02-01                 18080.26           209475.85
2   1953-03-01                 18080.26           208688.34
3   1953-04-01                 18122.25           209173.00
4   1953-05-01                 18143.25           208631.07
5   1953-06-01                 18143.25           207852.59
6   1953-07-01                 18185.25           208333.75
7   1953-08-01                 18206.25           207798.96
8   1953-09-01                 18227.25           208038.65
9   1953-10-01                 18227.25           207268.14
10  1953-11-01                 18206.25           207798.96
11  1953-12-01                 18206.25           207798.96

### Checking Null values

In [24]:
df.isnull().sum()

category                   0
Median Home Price (NSA)    0
CPI-Adjusted Price         0
dtype: int64

In [25]:
df.category = pd.to_datetime(df.category)

### Final dataset 
### Combining target dataset and df dataset

In [26]:
final_dataset = pd.merge(left=data, right=df, left_on='DATE', right_on='category',how='left')
final_dataset.head()

DATE  HNFSEPUSSA  HOUST1F  HSN1F  INTDSRUSM193N  LFACTTTTUSM657S  \
0 2000-01-01       311.0     1268    873           5.00         1.490972   
1 2000-02-01       299.0     1255    856           5.24         0.132849   
2 2000-03-01       313.0     1313    900           5.34        -0.015443   
3 2000-04-01       305.0     1275    841           5.50         0.222559   
4 2000-05-01       305.0     1230    857           5.71        -0.254289   

   MSACSR  NA000334Q    NASDAQCOM  PERMIT  PERMIT1  QUSR628BIS  RSAHORUSQ156S  \
0     4.3  2448892.0  4013.494000  1727.0   1277.0     95.3071           67.1   
1     4.3  2448892.0  4410.871000  1692.0   1241.0     95.3071           67.1   
2     4.3  2448892.0  4802.988261  1651.0   1253.0     95.3071           67.1   
3     4.4  2569266.0  3863.637368  1597.0   1192.0     96.7197           67.3   
4     4.4  2569266.0  3528.416818  1543.0   1182.0     96.7197           67.3   

    TTLCONS  UNRATE  CSUSHPISA   category  Median Home Price (NSA)  \
0  784940.0     4.0    100.551 2000-01-01                131559.13   
1  793737.0     4.1    101.339 2000-02-01                131318.85   
2  809459.0     4.0    102.127 2000-03-01                132827.82   
3  804766.0     3.8    102.922 2000-04-01                134096.51   
4  805005.0     4.0    103.678 2000-05-01                135144.14   

   CPI-Adjusted Price  
0           239289.53  
1           237445.83  
2           238210.25  
3           240345.10  
4           241940.32

In [27]:
final_dataset.isnull().sum()

DATE                       0
HNFSEPUSSA                 0
HOUST1F                    0
HSN1F                      0
INTDSRUSM193N              0
LFACTTTTUSM657S            0
MSACSR                     0
NA000334Q                  0
NASDAQCOM                  0
PERMIT                     0
PERMIT1                    0
QUSR628BIS                 0
RSAHORUSQ156S              0
TTLCONS                    0
UNRATE                     0
CSUSHPISA                  0
category                   0
Median Home Price (NSA)    0
CPI-Adjusted Price         0
dtype: int64

### Removing unnecessary column

In [28]:
final_dataset.drop(columns=['category'], inplace=True)

## Final dataset

In [29]:
final_dataset.head()

DATE  HNFSEPUSSA  HOUST1F  HSN1F  INTDSRUSM193N  LFACTTTTUSM657S  \
0 2000-01-01       311.0     1268    873           5.00         1.490972   
1 2000-02-01       299.0     1255    856           5.24         0.132849   
2 2000-03-01       313.0     1313    900           5.34        -0.015443   
3 2000-04-01       305.0     1275    841           5.50         0.222559   
4 2000-05-01       305.0     1230    857           5.71        -0.254289   

   MSACSR  NA000334Q    NASDAQCOM  PERMIT  PERMIT1  QUSR628BIS  RSAHORUSQ156S  \
0     4.3  2448892.0  4013.494000  1727.0   1277.0     95.3071           67.1   
1     4.3  2448892.0  4410.871000  1692.0   1241.0     95.3071           67.1   
2     4.3  2448892.0  4802.988261  1651.0   1253.0     95.3071           67.1   
3     4.4  2569266.0  3863.637368  1597.0   1192.0     96.7197           67.3   
4     4.4  2569266.0  3528.416818  1543.0   1182.0     96.7197           67.3   

    TTLCONS  UNRATE  CSUSHPISA  Median Home Price (NSA)  CPI-Adjusted Price  
0  784940.0     4.0    100.551                131559.13           239289.53  
1  793737.0     4.1    101.339                131318.85           237445.83  
2  809459.0     4.0    102.127                132827.82           238210.25  
3  804766.0     3.8    102.922                134096.51           240345.10  
4  805005.0     4.0    103.678                135144.14           241940.32

In [30]:
final_dataset.columns

Index(['DATE', 'HNFSEPUSSA', 'HOUST1F', 'HSN1F', 'INTDSRUSM193N',
       'LFACTTTTUSM657S', 'MSACSR', 'NA000334Q', 'NASDAQCOM', 'PERMIT',
       'PERMIT1', 'QUSR628BIS', 'RSAHORUSQ156S', 'TTLCONS', 'UNRATE',
       'CSUSHPISA', 'Median Home Price (NSA)', 'CPI-Adjusted Price'],
      dtype='object')

In [31]:
final_dataset = final_dataset[['DATE', 'HNFSEPUSSA', 'HOUST1F', 'HSN1F', 'INTDSRUSM193N',
       'LFACTTTTUSM657S', 'MSACSR', 'NA000334Q', 'NASDAQCOM', 'PERMIT',
       'PERMIT1', 'QUSR628BIS', 'RSAHORUSQ156S', 'TTLCONS', 'UNRATE', 'Median Home Price (NSA)', 
                  'CPI-Adjusted Price','CSUSHPISA']]

### Saving dataset to csv 

In [32]:
final_dataset.to_csv('final_dataset.csv', index= False)

# Interpretation of Column Names

#### CSUSHPISA: S&P/Case-Shiller U.S. National Home Price Index
#### HNFSEPUSSA: New One Family Homes for Sale in the United States
#### HOUST1F: New Privately-Owned Housing Units Started: Single-Family Units
#### HSN1F: New One Family Houses Sold: United States
#### INTDSRUSM193N: Interest Rates, Discount Rate for United States
#### LFACTTTTUSM657S: Active Population: Aged 15 and over: All Persons for United States
#### MSACSR: Monthly Supply of New Houses in the United States
#### NA000334Q: Gross Domestic Product
#### NASDAQCOM: NASDAQ Composite Index
#### PERMIT: New Privately-Owned Housing Units Authorized in Permit-Issuing Places: Total Units
#### PERMIT1: New Privately-Owned Housing Units Authorized in Permit-Issuing Places: Single-Family Units
#### QUSR628BIS: Real Residential Property Prices for United States
#### RSAHORUSQ156S: Homeownership Rate in the United States
#### TTLCONS: Total Construction Spending: Total Construction in the United States
#### UNRATE: Unemployment Rate
#### CPI (Adjusted Price)
#### Median Home Prices (NSA)